In [2]:
import aiohttp
import asyncio
import time
import json
import pandas as pd
from bs4 import BeautifulSoup
import lxml
import progressbar
from time import sleep

In [3]:
async def get_item_data(session, url):
    async with session.get(url) as resp:
        site_data = await resp.read()
        return site_data


async def main():
    
    with open(f'otomototabela.csv', 'w') as f:
        column_labels='auto'+'|cena'+'|rok_produkcji'+'|przebieg'+'|poj_silnika'+'|rodzaj_paliwa'+'\n'
        f.writelines(column_labels)
    ranges=[range(i*100+1,i*100+100) for i in range(60)]
    bar = progressbar.ProgressBar(maxval=len(ranges)).start()
    for idx, seq in enumerate(ranges):
        bar.update(idx)
        async with aiohttp.ClientSession() as session:
            tasks = []
            for number in seq:
                url = f'https://www.otomoto.pl/osobowe/page={number}'
                #print(url)
                tasks.append(asyncio.ensure_future(get_item_data(session, url)))
            oto_moto_page = await asyncio.gather(*tasks)

            with open(f'otomototabela.csv', 'a') as f:
                for page in oto_moto_page:
                    page=BeautifulSoup(page,features='html5')
                    for article in page.find_all('article',class_='ooa-15j3s1f e1b25f6f0'):
                        try:
                            f.writelines(article.h2.a.text+"|"+article.find_all('div')[5].div.span.text+
                                         "|"+article.div.ul.find_all('li')[0].text+"|"+article.div.ul.find_all('li')[1].text+
                                         "|"+article.div.ul.find_all('li')[2].text+"|"+article.div.ul.find_all('li')[3].text+
                                         '\n')
                        except:
                            continue

start_time=time.time()
await main()
print("--- %s seconds ---" % (time.time() - start_time))

 98% (59 of 60) |####################### | Elapsed Time: 0:27:48 ETA:   0:00:27

--- 1697.158680677414 seconds ---


In [4]:
auta=pd.read_csv("otomototabela.csv",encoding = 'Windows-1252',sep="|",on_bad_lines='skip')

In [6]:
auta

,auto,cena,rok_produkcji,przebieg,poj_silnika,rodzaj_paliwa
0,Seat Leon,78 000 PLN,2018,81 749 km,1 498 cm3,Benzyna
1,Opel Zafira Tourer 2.0 CDTI Automatik,37 999 PLN,2014,242 000 km,1 956 cm3,Diesel
2,Hyundai Elantra,59 000 PLN,2018,50 573 km,1 591 cm3,Benzyna
3,Seat Leon,78 000 PLN,2018,81 749 km,1 498 cm3,Benzyna
4,Opel Zafira Tourer 2.0 CDTI Automatik,37 999 PLN,2014,242 000 km,1 956 cm3,Diesel
...,...,...,...,...,...,...
26465,Seat Ibiza 1.4 16V,16 900 PLN,2009,178 000 km,1 390 cm3,Benzyna
26466,Audi Q7 45 TDI mHEV Quattro S Line Tiptr,384 000 PLN,2022,10 km,2 967 cm3,Diesel
26467,Opel Astra IV 1.4 T Edition 150,34 800 PLN,2013,180 000 km,1 364 cm3,Benzyna
26468,Audi A6,85 900 PLN,2016,182 000 km,1 968 cm3,Diesel


In [7]:
auta.drop_duplicates(['auto', 'przebieg','cena'],inplace=True)

In [8]:
auta

,auto,cena,rok_produkcji,przebieg,poj_silnika,rodzaj_paliwa
0,Seat Leon,78 000 PLN,2018,81 749 km,1 498 cm3,Benzyna
1,Opel Zafira Tourer 2.0 CDTI Automatik,37 999 PLN,2014,242 000 km,1 956 cm3,Diesel
2,Hyundai Elantra,59 000 PLN,2018,50 573 km,1 591 cm3,Benzyna
582,Seat Toledo 1.6 Stella,4 999 PLN,2002,220 000 km,1 595 cm3,Benzyna
583,Volkswagen Golf,7 000 PLN,2000,262 332 km,1 598 cm3,Benzyna
584,Volkswagen Sharan 2.0 TDI BMT Highline DSG,79 400 PLN,2018,126 150 km,1 968 cm3,Diesel
2256,Toyota Yaris,30 500 PLN,2013,83 379 km,998 cm3,Benzyna
8573,Opel Insignia,74 000 PLN,2018,84 961 km,1 956 cm3,Diesel
9374,Opel Mokka 1.4 T Cosmo S&S 4x4,47 900 PLN,2013,173 000 km,1 364 cm3,Benzyna
9947,Chrysler Town & Country 4.0 Limited,30 000 PLN,2008,150 000 km,3 952 cm3,Benzyna


In [278]:
auta['Marka'] = auta['auto'].str.split(' ', 1, expand=True)[0]

In [279]:
auta

,auto,model,cena,rok_produkcji,przebieg,poj_silnika,rodzaj_paliwa,Marka
0,Peugeot 207 1.4 Presence,"1.4 -73 KM Klimatyzacja, Super Stan, Du¿y Serw...",14 999 PLN,2010,179 000 km,1 360 cm3,Benzyna,Peugeot
1,Porsche Macan S Diesel,Macan S. Salon Polska. Bezwypadkowy. 1 w³aœcic...,245 000 PLN,2017,70 728 km,2 967 cm3,Diesel,Porsche
2,Audi A3 1.5 TFSI cylinder on demand Sportback ...,"Pierwszy w³aœciciel , Serwisowany, Piêkny kolor",93 500 PLN,2019,105 000 km,1 498 cm3,Benzyna,Audi
3,Jeep Grand Cherokee Gr 3.6 V6 Summit,"Jeep Grand Cherokee Overland 2015 3,6l",89 000 PLN,2015,102 000 km,3 604 cm3,Benzyna,Jeep
4,Toyota Aygo 1.0 VVT-i Sol,1.0 Klima 5 drzwi ALU Niski Przebieg 4.5l/100 ...,15 999 PLN,2011,145 000 km,998 cm3,Benzyna,Toyota
...,...,...,...,...,...,...,...,...
2656,Citroën C3 1.2 PureTech Shine S&S,Citroen C3 110 kM Pierwszy w³aœciciel,50 700 PLN,2018,45 000 km,1 199 cm3,Benzyna,Citroën
2657,Renault Master ENERGY dCi 150 L2H3 VA,"2,3 50tys km Ful NAVI RVM Karera czuj parko TY...",89 999 PLN,2019,390 215 km,2 299 cm3,Diesel,Renault
2658,Peugeot 3008 1.6 HDi Style,KAMERA !!! / SERWIS !!! / PANORAMA !!! /,38 900 PLN,2014,205 000 km,1 560 cm3,Diesel,Peugeot
2659,Mercedes-Benz GLE Coupe 350 d 4-Matic,350 4Matic COUPE VAT23%,258 300 PLN,2017,126 000 km,2 987 cm3,Diesel,Mercedes-Benz


In [280]:
auta['wersja'] = auta['auto'].str.split(' ', 2, expand=True)[1]

In [281]:
filterseries= auta['wersja']=="Seria"

In [282]:
auta.where(filterseries).dropna()['wersja'].unique()

array(['Seria'], dtype=object)

In [283]:
auta.loc[filterseries,'wersja']=auta['auto'].str.split(' ', 4, expand=True)[1]+" "+auta['auto'].str.split(' ', 4, expand=True)[2]

In [284]:
auta.where(filterseries).dropna()

,auto,model,cena,rok_produkcji,przebieg,poj_silnika,rodzaj_paliwa,Marka,wersja
16,BMW Seria 7 750i,BMW 750i xDrive - stan perfekcyjny,187 000 PLN,2015.0,92 000 km,4 395 cm3,Benzyna,BMW,Seria 7
35,BMW Seria 5,"530d 218Hp M Pakiet Fabryczny, Head Up, Komfor...",38 900 PLN,2005.0,320 000 km,2 993 cm3,Diesel,BMW,Seria 5
52,BMW Seria 2 218d GT Luxury Line,Bmw 218d GT Luxury Line mo¿liwa zamiana,67 000 PLN,2016.0,275 000 km,1 995 cm3,Diesel,BMW,Seria 2
53,BMW Seria 4 M440d xDrive Coupe,"CenaDoKoncaRoku Dostêpne od rêki, nowe M4 Cera...",376 900 PLN,2022.0,10 km,2 993 cm3,Diesel,BMW,Seria 4
62,BMW Seria 5 540d xDrive Luxury Line,Model 2019 320KM IDEA£ pe³na faktura,218 940 PLN,2018.0,133 000 km,2 993 cm3,Diesel,BMW,Seria 5
...,...,...,...,...,...,...,...,...,...
2578,BMW Seria 5 520d,"* 2,0 177KM * LIFT * Sedan * Manual * NAVI * S...",29 800 PLN,2009.0,232 881 km,1 995 cm3,Diesel,BMW,Seria 5
2590,BMW Seria 7 730d xDrive,"BMW G11 730D xDrive, pierwszy w³aœciel.",259 000 PLN,2019.0,130 000 km,2 993 cm3,Diesel,BMW,Seria 7
2645,BMW Seria 5 520d xDrive MHEV Business Edition,Salon Polska FV23% gwarancja do 2024 cena Brutto,197 900 PLN,2021.0,26 900 km,1 995 cm3,Diesel,BMW,Seria 5
2651,BMW Seria 5 M550d xDrive,M550d xDrive pakiet serwisowy do 03/2023,309 000 PLN,2018.0,49 832 km,2 993 cm3,Diesel,BMW,Seria 5


In [285]:
filterbmw= auta['Marka']=="BMW"

In [286]:
auta.where(filterbmw).dropna()['wersja'].unique()

array(['Seria 7', 'Seria 5', 'X3', 'Seria 2', 'Seria 4', '5GT', 'M3',
       'Seria 3', 'X5', 'X7', 'Seria 1', 'M8', 'X1', 'X4', 'Seria 8',
       'X6', 'X2', 'M2', '3GT', 'M4', 'Seria 6', 'M5'], dtype=object)

In [287]:
filteropel= auta['Marka']=="Opel"

In [289]:
auta.where(filteropel).dropna()['wersja'].unique()

array(['Astra', 'Corsa', 'Insignia', 'Mokka', 'Combo', 'Grandland',
       'Antara', 'Meriva', 'Zafira', 'Adam', 'Vectra', 'Vivaro', 'Signum',
       'Crossland', 'Omega', 'Agila'], dtype=object)

In [290]:
filterskoda= auta['Marka']=="Skoda"

In [291]:
auta.where(filterskoda).dropna()['wersja'].unique()

array(['Octavia', 'Fabia', 'Superb', 'Kamiq', 'RAPID', 'Kodiaq',
       'Roomster', 'Karoq', 'Scala', 'Yeti', 'Citigo'], dtype=object)

In [293]:
filteraudi= auta['Marka']=="Audi"

In [294]:
auta.where(filteraudi).dropna()['wersja'].unique()

array(['A3', 'A4', 'Q2', 'A6', 'A7', 'A5', 'Q5', 'S3', 'A8', 'A1', 'Q7',
       'R8', 'TT', 'Q8', 'S5', 'S8', 'S6', 'RS3', 'RS5', 'S7', 'S1',
       'SQ7', 'Q3', 'SQ5', 'RS4', 'RS', 'S4'], dtype=object)

In [295]:
filterkia= auta['Marka']=="Kia"

In [296]:
auta.where(filterkia).dropna()['wersja'].unique()

array(['Sportage', 'Stinger', 'Ceed', 'Optima', 'Sorento', 'Carens',
       'Rio', 'Venga', "Pro_cee'd", 'Stonic', 'Picanto', 'XCeed', 'Soul',
       'Carnival'], dtype=object)

In [297]:
filtervw= auta['Marka']=="Volkswagen"

In [298]:
auta.where(filtervw).dropna()['wersja'].unique()

array(['T-Cross', 'Tiguan', 'Taigo', 'T-Roc', 'Beetle', 'Passat',
       'Arteon', 'Sharan', 'Golf', 'Jetta', 'Touran', 'Scirocco', 'up!',
       'Transporter', 'CC', 'Polo', 'Multivan', 'Touareg', 'New',
       'Crafter', 'California', 'Caddy', 'Eos', 'Fox'], dtype=object)

In [301]:
filterrenault= auta['Marka']=="Renault"

In [302]:
auta.where(filterrenault).dropna()['wersja'].unique()

array(['Talisman', 'Kadjar', 'Megane', 'Clio', 'Kangoo', 'Captur',
       'Scenic', 'Koleos', 'Trafic', 'Twingo', 'Laguna', 'Espace',
       'Grand', 'Arkana', 'Modus', 'Master'], dtype=object)